In [2]:
# Installations

#! pip install bson
#! pip install pandas
#! pip install dirtyjson

In [3]:
# Imports
import pandas as pd
import json, dirtyjson
import csv
from io import StringIO

In [4]:
#Configuration

file_path = 'data/4chan_online.csv'

In [5]:
df = pd.read_csv(file_path, usecols=['_id','posts', 'last_modified', 'archived', 'post_time_UTC', 'scraped_time_UTC', 'board_code'])

In [6]:
posts = []
for i in range(len(df['posts'])):
    line = df['posts'].iloc[i]
    try:
        json_line = json.loads(line)
        posts.append(json_line)
    except json.JSONDecodeError as e:
        #print(f"Error decoding JSON: {e}")
        continue

In [7]:
# Users
users = []
for i in range(len(posts)):
    for j in range(len(posts[i])):
        p = posts[i][j]
        if 'name' in p.keys(): 
            users.append(posts[i][j]['name'])
totalUsers = len(users)
from collections import Counter
c = Counter(users)
commonUsers = c.most_common()[:40]
for user in commonUsers:
    print(user[0] + ' & ' + str(round((user[1]/totalUsers)*100, 2)))

Anonymous & 97.47
Minifig & 0.04
Translatorfag & 0.03
sage & 0.03
Lanu & 0.03
justingod & 0.03
OP & 0.02
Greeble & 0.02
RumicWorldStoryTime & 0.02
cANON & 0.02
QM & 0.02
Dannyanon & 0.02
Anonymous Mogul & 0.02
AiWeaver & 0.02
MedivhQM & 0.02
DrDragonfagQM & 0.02
No&#039;body & 0.02
ar & 0.02
GrandDragonQM & 0.02
Mulrog & 0.02
Contolist & 0.02
Trips Namefag & 0.02
Piggy & 0.01
DOT DOT DOT & 0.01
Builder_Referee & 0.01
Gacha guy & 0.01
fe2fucker & 0.01
Billy & 0.01
Cult of Passion & 0.01
Magister & 0.01
Our-Grandfather & 0.01
Captain Kohai (Kancho Kohai) & 0.01
Squig-Anon & 0.01
ReptoidQM & 0.01
TempQM & 0.01
Lesches & 0.01
@non & 0.01
Krantagarh & 0.01
Goggled Anon & 0.01
Big Hat Steve & 0.01


In [8]:
#Posts length per board
import re

boardPosts = {}
for i in range(len(df['posts'])):
    line = df['posts'].iloc[i]
    board = df['board_code'].iloc[i]
    try:
        json_line = json.loads(line)
        if board in boardPosts.keys():
            boardPosts[board].append(json_line)
        else:
            boardPosts[board] = [json_line]
    except json.JSONDecodeError as e:
        #print(f"Error decoding JSON: {e}")
        continue

In [9]:
postMsgs = {}
for k,discussions in boardPosts.items():
    ls = []
    for discussion in discussions:
        for replies in discussion:
            if 'com' in replies.keys():
                com = replies['com']
                com = re.sub('<[^<]+?>', '', com)
                com = re.sub(r'http\S+', '', com)
                ls.append(com)       
    ls.append(len(com))
    postMsgs[k] = ls


In [9]:
postLengths = {}
for k,v in postMsgs.items():
    ls = []
    for p in v:
        cc = len(str(p))
        ls.append(cc)
    postLengths[k] = ls

In [13]:
import numpy as np
dicMean = {}
dicMedian = {}
for k,v in postLengths.items():
    dicMean[k] = np.mean(v)
    dicMedian[k] = np.median(v)

In [14]:
sortedDicMean = {k: v for k, v in sorted(dicMean.items(), key=lambda item: item[1])}
sortedDicMedian = {k: v for k, v in sorted(dicMedian.items(), key=lambda item: item[1])}
sortedDicMedian.values()

dict_values([20.0, 37.0, 38.0, 39.0, 40.0, 40.0, 42.0, 43.0, 44.0, 45.0, 47.0, 49.0, 51.0, 52.0, 53.0, 55.0, 56.0, 57.0, 58.0, 59.0, 60.0, 60.0, 60.0, 61.0, 62.0, 62.0, 64.0, 67.0, 68.0, 68.0, 69.0, 70.0, 72.0, 72.0, 75.0, 75.0, 78.0, 79.0, 82.0, 85.0, 87.0, 87.0, 87.0, 87.0, 88.0, 89.0, 89.0, 90.0, 90.0, 91.0, 94.0, 94.0, 96.0, 102.0, 103.0, 106.0, 107.0, 107.0, 108.0, 112.0, 112.0, 114.0, 116.0, 118.0, 118.0, 120.0, 122.0, 122.0, 124.0, 125.0, 127.0, 129.0, 131.0, 144.0, 157.0, 163.0, 177.0, 210.0])

In [18]:
#! pip install -q transformers
#!pip install tf-keras

In [10]:
from transformers import pipeline
from transformers import AutoTokenizer
import warnings
warnings.filterwarnings("ignore")
model = f"cardiffnlp/twitter-roberta-base-sentiment-latest"
tokenizer = AutoTokenizer.from_pretrained(model)
sentiment_task = pipeline("sentiment-analysis", model=model, tokenizer=tokenizer)

C:\projects\4chan\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Some weights of the model checkpoint at cardiffnlp/twitter-roberta-base-sentiment-latest were not used when initializing RobertaForSequenceClassification: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
- This IS expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [11]:
import random
sampledMsgs = dict.fromkeys(postMsgs.keys())
for k,v in postMsgs.items():
    if len(v) > 100:
        sampledMsgs[k] = random.sample(postMsgs[k], 100)
    else:
        sampledMsgs[k] = postMsgs[k]

In [12]:
# Sentiment Analysis
dictSentiments = dict.fromkeys(sampledMsgs.keys(),[])
#data = ["I love you", "I hate you"]
for k,v in sampledMsgs.items():
    ls = []
    for com in v:
        try:
            ls.append(sentiment_task(com))
        except:
            continue
    dictSentiments[k] = ls

In [26]:
dictSentiLabels = dict.fromkeys(dictSentiments.keys(),[])
for k,v in dictSentiments.items():
    labels = [] 
    for lb in v:
        labels.append(lb[0]['label']) 
    dictSentiLabels[k] = labels


In [28]:
for k,v in dictSentiLabels.items():
    #print(k, v.count('positive'), v.count('neutral'), v.count('negative'))
    continue

In [ ]:
#Toxicity
#!pip install detoxify
from detoxify import Detoxify
dictToxicity = dict.fromkeys(sampledMsgs.keys(), [])
for k,v in sampledMsgs.items():
    ls = []
    for msg in v:
        try:
            results = Detoxify('original').predict(msg)
        except:
            continue
        ls.append(results)
    dictToxicity[k] = ls

In [ ]:
dictToxicity['vt'][0]

In [84]:
#Total boards
len(set(df['board_code']))

78

In [85]:
#all posts across all boards
allposts = []
for i in range(len(df['posts'])):
    allposts.append(df['posts'][i].count('}'))
print('Count of all posts: ', len(allposts))

#all posts across all boards with Anonymous user
AnonymousPosts = 0
for i in range(len(df['posts'])):
    val = df['posts'][i].count('Anonymous')
    if  val > 0:
        AnonymousPosts += val
print('Count of all posts vs Anonymous posts: ', sum(allposts), 'vs', AnonymousPosts)

#Key as no of posts - value no of discussions
postsBins = {}
for pp in allposts:
    if pp in postsBins.keys():
        postsBins[pp] += 1
    else:
        postsBins[pp] = 1
print('total number of discussions: ', sum(postsBins.values()))
print('discussions with unique no. of posts: ', len(postsBins.values()))

Count of all posts:  11642
Count of all posts vs Anonymous posts:  1032012 vs 1002530
total number of discussions:  11642
discussions with unique no. of posts:  588


In [86]:
#Seperating discussions with number of posts into fixed bins 
# [0 posts, less than 100 posts, between 100-199 posts, between 200-299 posts, between 300-399 posts, 400+ posts]
hundbins = {0: 0, 100: 0, 200: 0, 300: 0, 400: 0, 500:0}
for k, v in postsBins.items():
    if v == 1:
        hundbins[0] += 1
    if v < 100:
        hundbins[100] += 1
    if v >= 100 and v < 200:
        hundbins[200] += 1
    if v >= 200 and v < 300:
        hundbins[300] += 1
    if v >= 300 and v < 400:
        hundbins[400] += 1
    if v >= 400:
        hundbins[500] += 1
hundbins

{0: 141, 100: 564, 200: 15, 300: 4, 400: 3, 500: 2}

In [87]:
tot = sum(hundbins.values())
hundbinpc = []
for k, v in hundbins.items():
    hundbinpc.append((v/tot)*100)
hundbinpc

[19.34156378600823,
 77.36625514403292,
 2.05761316872428,
 0.5486968449931412,
 0.411522633744856,
 0.2743484224965706]

In [88]:
boards = df.groupby('board_code')
board_codes = list(set(df['board_code']))
len(board_codes)

scores = []
for bc in board_codes:
    scores.append(boards.get_group(bc).count().iloc[0])

scores.sort()    
#scores

Posts analysis

In [11]:
import re
import json

def cleanJSON(dirty_json):
    regex_replace = [(r"([ \{,:\[])(u)?'([^']+)'", r'\1"\3"'), (r" False([, \}\]])", r' false\1'), (r" True([, \}\]])", r' true\1')]
    for r, s in regex_replace:
        dirty_json = re.sub(r, s, dirty_json)
    dirty_json.replace('\\', '')
    clean_json = json.loads(dirty_json)
    return clean_json

In [12]:
import re
# as per recommendation from @freylis, compile once only
CLEANR = re.compile('<.*?>') 

def cleanhtml(raw_html):
  cleantext = re.sub(CLEANR, '', raw_html)
  return cleantext

In [89]:
# Discussion titles vs no. of posts

dicTitlesPosts = {}
for d in range(len(df['posts'])):
    #print(d)
    currPost = df['posts'][d]
    board = df['board_code'][d]
    #specific dirty cases
    count = currPost.count('}')
    ind1 = currPost.find("sub")+6
    ind2 = currPost[ind1:].find("\"")
    title = board+"-"+currPost[ind1:ind1+ind2]
    if title not in dicTitlesPosts.keys():
        dicTitlesPosts[title] = count


In [90]:
sorted_dict = dict(sorted(dicTitlesPosts.items(), key=lambda item: item[1], reverse = True))

In [91]:
i = 0
for k,v in sorted_dict.items():
    print(k,v)
    i += 1
    if i == 20:
        break

vmg-Dokkan Battle 1562
h-hdg/ Hentai Diffusion General #536 1359
qst-The King &amp; Queen of /QST/: Nominations, Qualifications, &amp; Preparations 1339
vm-/cs2g/CS2/CSGO/Counter-Strike Esports discussion ESL Pro League XIX edition 1293
qst-QTG: Easter Edition 1260
sp-/cyc/ - Giro d&#039;Italia - Stage 1 1155
qst-HERO(&#039;s party member) QUEST 1038
u-General Yuri Discussion Thread 1005
vmg-Fire Emblem Heroes 1003
vm-/Bombergirl/ General「ボンバーガール」 997
qst-Warlords of Chaos #1 981
vmg-Genshin Impact 925
u-Recent Yuri Releases 913
vg-/@/ - The iDOLM@STER General 912
vmg-Blue Archive JP 893
jp-DJT - Daily Japanese Thread #3855 884
vmg-Aether Gazer 878
vg-/vn/ - Completed Edition 870
vg-/tekgen/ - Tekken General 858
jp-hololive 843


In [132]:
# User names
import warnings
warnings.filterwarnings("ignore")

users = []
for i in range(len(df['posts'])):
    print(df['posts'][2])
    posts = json.loads(df['posts'][i])
    for post in posts:
        users.append(post['name'])

[{"no":1,"sticky":1,"closed":1,"now":"080720Fri06:50:24","name":"Anonymous","sub":"Welcome to vmg  VidyaMobile Games","com":"vmg is a place to discuss all types of mobile games, including both Android and iOS, phones and tablets Please note that Nintendo and Sony handhelds are considered consoles, and that threads pertaining to those platforms and their gaming titles should be posted elsewherebrnbrnDoes this mean discussion of mobile games is banned on other video game boards span style="fontsize:15pxfontweight:bold"Nospan vmg is just a separate board specifically focused on mobile games where discussions about your favorite games can thrivebrnbrnPlease familiarize yourself with a href="https:www4channelorgrulesvmg"the rulesa and remember to a href="https:www4channelorgfaqspoiler"use the spoiler function where appropriateabrnnp style="fontsize:15pxfontweight:bold"Please note that, like v, quotGeneralsquotlongterm, oneaftertheother, recurring threads about a specific game are not permit

JSONDecodeError: Expecting ',' delimiter: line 1 column 494 (char 493)

In [29]:
# User names
import warnings
warnings.filterwarnings("ignore")

count = 0
hasImage = 0
msg = []
for i in range(len(df['posts'])):
    a = pd.read_json(df['posts'][i])
    a = a['name']
    print(a.head())
    break


0    Anonymous
Name: name, dtype: object


In [17]:
count, hasImage

(3038562, 3038562)

In [358]:
msg[0]

IndexError: list index out of range

In [331]:
count = 0
hasImage = 0
msg = []
for i in range(len(df['posts'])):
    a = df['posts'][i]
    a = cleanhtml(a)
    a = cleanJSON(a)
    a = json.dumps(a)
    dfposts = dirtyjson.loads(a)
    for j in range(len(dfposts)):
        #print(dfposts[j]['filename'])
        #print(dfposts[j]['com'])
        count += 1
        
        if 'filename' in dfposts[j]:
            hasImage += 1
        if 'com' in dfposts[j]:
            msg.append(dfposts[j]['com'])

JSONDecodeError: Invalid \escape: line 1 column 56469 (char 56468)

In [317]:
count, hasImage, msg

(20,
 20,
 ['Please post more pics like this. Thanks anons.',
  '<a href="#p2185924" class="quotelink">&gt;&gt;2185924</a><br>ill post some more if this is still up. ive been awake 23hrs and am going to bed. also ask in the ak anime thread on <span class="deadlink">&gt;&gt;&gt;/k/47495512</span>',
  '<a href="#p2185924" class="quotelink">&gt;&gt;2185924</a>',
  '<a href="#p2185924" class="quotelink">&gt;&gt;2185924</a>'])

In [1]:
# Sentiment analysis of posts

len(postMsgs)

NameError: name 'postMsgs' is not defined

In [ ]:
#! pip install -q transformers
#! pip install tensorflow
#!pip install -q torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu118
#! pip install -q distutils
!pip uninstall tensorflow
!pip install tensorflow==2.2.0

In [28]:
import torch
from transformers import pipeline
sentiment_pipeline = pipeline("sentiment-analysis")
data = ["I love you", "I hate you"]
sentiment_pipeline(data)

No model was supplied, defaulted to distilbert/distilbert-base-uncased-finetuned-sst-2-english and revision af0f99b (https://huggingface.co/distilbert/distilbert-base-uncased-finetuned-sst-2-english).
Using a pipeline without specifying a model name and revision in production is not recommended.
C:\projects\4chan\Lib\site-packages\huggingface_hub\file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


RuntimeError: At least one of TensorFlow 2.0 or PyTorch should be installed. To install TensorFlow 2.0, read the instructions at https://www.tensorflow.org/install/ To install PyTorch, read the instructions at https://pytorch.org/.

In [15]:
# Discussion titles vs no. of posts

titles_posts = {}
for d in range(len(df['posts'])):
    posts = json.loads(df['posts'][2000])
    postscount = len(posts)
    for post in posts:
        dfpost = pd.DataFrame.from_dict(post, orient='index')
        #print(len(dfpost))
        #print(dfpost)
    break




In [30]:
len(df['posts'])

11642